In [13]:
!pip install nest_asyncio

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
import asyncio
import time
import nest_asyncio

nest_asyncio.apply()

# Synchronous function
def sync_worker(number):
    print(f'Sync Worker {number} starts')
    time.sleep(1)
    print(f'Sync Worker {number} ends')
    return f'Sync Worker {number}'

# Asynchronous function
async def async_worker(number):
    print(f'Async Worker {number} starts')
    await asyncio.sleep(1)
    print(f'Async Worker {number} ends')
    return f'Async Worker {number}'

# Synchronous execution
start_time = time.time()
for i in range(3):
    sync_worker(i)
print(f'Synchronous execution time: {time.time() - start_time}')

# Asynchronous execution
start_time = time.time()
loop = asyncio.get_event_loop()
tasks = [async_worker(i) for i in range(3)]
loop.run_until_complete(asyncio.gather(*tasks))
print(f'Asynchronous execution time: {time.time() - start_time}')


Sync Worker 0 starts
Sync Worker 0 ends
Sync Worker 1 starts
Sync Worker 1 ends
Sync Worker 2 starts
Sync Worker 2 ends
Synchronous execution time: 3.0164549350738525
Async Worker 0 starts
Async Worker 1 starts
Async Worker 2 starts
Async Worker 0 ends
Async Worker 1 ends
Async Worker 2 ends
Asynchronous execution time: 1.006666898727417


In [4]:
from fastapi import FastAPI
import time

app = FastAPI()

@app.get("/sleep/{seconds}")
async def sleep_for_seconds(seconds: int):
    time.sleep(seconds)
    return {"Success": True, "SleptFor": seconds}


In [16]:
# =============================================================================
# DB Connectivity using SQLite3
# =============================================================================
import sqlite3
from sqlite3 import Error

def create_connection():
    conn = None;
    try:
        conn = sqlite3.connect(":memory:") # creates a memory-based temporary database for demo
        print(f'successful connection with sqlite version {sqlite3.version}')
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()

create_connection()


successful connection with sqlite version 2.6.0


In [17]:
# =============================================================================
# DB Connectivity using SQLite3 -- Example 2
# =============================================================================
import sqlite3

# Connect to the database
conn = sqlite3.connect('example.db')

# Create a cursor object
cursor = conn.cursor()

# # Create a table
cursor.execute('''CREATE TABLE users
                  (id INTEGER PRIMARY KEY, name TEXT, email TEXT)''')

# Insert data into the table
cursor.execute("INSERT INTO users (name, email) VALUES (?, ?)", ('John', 'john@example.com'))

# Fetch data from the table
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the database connection
conn.close()

(1, 'John', 'john@example.com')


In [8]:
# =============================================================================
# HogwartsMember
# =============================================================================
from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional

app = FastAPI()

class HogwartsMember(BaseModel):
    name: str
    house: Optional[str] = None
    year: Optional[int] = None

members = [
    {"name": "Harry Potter", "house": "Gryffindor", "year": 5},
    {"name": "Hermione Granger", "house": "Gryffindor", "year": 5},
]

@app.get("/hogwarts/members/{member_id}")
def get_member(member_id: int):
    if 0 <= member_id < len(members):
        return members[member_id]
    return {"error": "Member not found"}

@app.post("/hogwarts/members")
def add_member(member: HogwartsMember):
    members.append(member.dict())
    return {"success": True, "member_id": len(members) - 1}

# =============================================================================
# Test cases
# =============================================================================
from fastapi.testclient import TestClient
import unittest

client = TestClient(app)

class TestHogwarts(unittest.TestCase):
    def test_get_member(self):
        response = client.get("/hogwarts/members/0")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json(), {"name": "Harry Potter", "house": "Gryffindor", "year": 5})

    def test_get_member_not_found(self):
        response = client.get("/hogwarts/members/999")
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json(), {"error": "Member not found"})

    def test_add_member(self):
        response = client.post("/hogwarts/members", json={"name": "Ron Weasley", "house": "Gryffindor", "year": 5})
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json(), {"success": True, "member_id": 2})

    def test_add_member_no_house(self):
        response = client.post("/hogwarts/members", json={"name": "Luna Lovegood", "year": 4})
        self.assertEqual(response.status_code, 200)
        self.assertEqual(response.json(), {"success": True, "member_id": 3})
        

response = client.get("/hogwarts/members/0")
print(response.status_code) # 200
print(response.json()) # {'name': 'Harry Potter', 'house': 'Gryffindor', 'year': 5}       

200
{'name': 'Harry Potter', 'house': 'Gryffindor', 'year': 5}


In [9]:
from pydantic import BaseModel
from typing import Optional

class HogwartsMember(BaseModel):
    name: str
    house: Optional[str] = None
    year: Optional[int] = None

new_member = HogwartsMember(name="Draco Malfoy", house="Slytherin", year=5)

response = client.post("/hogwarts/members", json=new_member.dict())
print(response.json())  # Should print: {'success': True, 'member_id': <new_member_id>}

# http://127.0.0.1:8000/hogwarts/members/2

{'success': True, 'member_id': 2}


/var/folders/ww/y8f04l7d767f4mpjwfq0lt8w0000gn/T/ipykernel_57192/933283442.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  response = client.post("/hogwarts/members", json=new_member.dict())
/var/folders/ww/y8f04l7d767f4mpjwfq0lt8w0000gn/T/ipykernel_57192/802263483.py:28: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.3/migration/
  members.append(member.dict())


In [10]:
from fastapi import FastAPI, WebSocket

app = FastAPI()

@app.websocket("/ws")
async def websocket_endpoint(websocket: WebSocket):
    await websocket.accept()
    while True:
        data = await websocket.receive_text()
        await websocket.send_text(f"data : {data}")


In [11]:
class ConnectionManager:
    def __init__(self):
        self.connections: Dict[int:List[WebSocket]] = dict()

    async def connect(self, websocket: WebSocket, room: int):
        await websocket.accept()
        if room not in self.connections:
            self.connections[room] = list()
        self.connections[room].append(websocket)

    async def broadcast(self, data: str, room: int):
        for connection in self.connections[room]:
            await connection.send_text(data)
